In [ ]:
import os
import pandas as pd
import qiime2 as q2
from biom import Table, load_table
from qiime2.plugins.diversity.methods import alpha
from qiime2.plugins.feature_table.methods import rarefy
from qiime2.plugins.diversity.methods import alpha_phylogenetic_alt


In [3]:
# rar depth (hard set to 5000)
rar_depth = 5000
# tables for import
tables_ = {'fecal':'../data/split-data/Baby-Feces',
            'oral':'../data/split-data/Baby-Mouth',
            'skin':'../data/split-data/Baby-Right_Forearm'}
# tree for import
tree = '../data/processed-data/tree.qza' 
tree = q2.Artifact.load(tree)
# get each table and run alpha's
for k_, path_ in tables_.items():
    print('Starting: %s' % k_)
    # get table(s)
    table_ = load_table(os.path.join(path_, 'table.biom'))
    mf_ = pd.read_csv(os.path.join(path_, 'metadata.tsv'),
                      sep='\t', index_col=0)
    mf_.columns = [col.replace('old_','') for col in mf_.columns]
    table_ = pd.DataFrame(table_.matrix_data.toarray(),
                          table_.ids('observation'),
                          table_.ids()).T
    tq2able_ = q2.Artifact.import_data('FeatureTable[Frequency]', table_)
    print('Rarefy-depth %i' % rar_depth)
    # run rare
    table_rar = rarefy(tq2able_, rar_depth).rarefied_table
    table_rar.save(os.path.join(path_, 'rarefy-table.qza'))
    # run alphas
    shannon_alpha = alpha(table_rar, 'shannon').alpha_diversity
    faiths_alpha = alpha_phylogenetic_alt(table_rar,
                                          tree,
                                          'faith_pd').alpha_diversity
    # save qza
    out_ = '../data/alpha-diversity/%s' % (k_)
    shannon_alpha.save(os.path.join(out_, 'shannon-alpha.qza'))
    faiths_alpha.save(os.path.join(out_, 'faiths-alpha.qza'))
    # save tsv + metadata for other purposes
    shannon_alpha = shannon_alpha.view(q2.Metadata).to_dataframe()
    faiths_alpha = faiths_alpha.view(q2.Metadata).to_dataframe()
    shannon_alpha = pd.concat([shannon_alpha,
                               mf_.reindex(shannon_alpha.index)],
                              axis=1)
    faiths_alpha = pd.concat([faiths_alpha,
                              mf_.reindex(faiths_alpha.index)],
                             axis=1)
    # write tsv
    shannon_alpha.to_csv(os.path.join(out_, 'shannon-alpha.tsv'), sep='\t')
    faiths_alpha.to_csv(os.path.join(out_, 'faiths-alpha.tsv'), sep='\t')


Starting: fecal
Rarefy-depth 5000
Starting: oral
Rarefy-depth 5000
Starting: skin
Rarefy-depth 5000
